In [1]:
import eikon as ek  # the Eikon Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import configparser as cp
cf.set_config_file(offline=True)  # set the plotting mode to offline

from numpy import where
from numpy import unique
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LinearRegression
import pylab as plt
import warnings
import configparser
from datetime import datetime

In [2]:
cfg = cp.ConfigParser()
cfg.read('../refinitiv.cfg')
ek.set_app_key(cfg['eikon']['app_id'])

In [3]:
#Pairs
ric = "IBM.N"
fields = ["TR.CommonName",
        "TR.HeadquartersCountry",
        "TR.ExchangeName",
        "TR.TRBCIndustryGroup",
        "TR.GICSSector",
        "TR.TickerSymbol",
        "TR.PriceMoPriceCurrency",
        "TR.CompanyMarketCap(scale=6, curn='USD')",
        "TR.EV(scale=6, curn='USD')"]
df, err = ek.get_data(ric,fields )
df

,Instrument,Company Common Name,Country of Headquarters,Exchange Name,TRBC Industry Group Name,GICS Sector Name,Ticker Symbol,Price Currency,Company Market Cap,Enterprise Value (Daily Time Series)
0,IBM.N,International Business Machines Corp,United States of America,"NEW YORK STOCK EXCHANGE, INC.",Software & IT Services,Information Technology,IBM,US Dollar,118858.048209,161146.048209


In [4]:
peer_rics, err = ek.get_data("Peers({})".format(ric), ["TR.PrimaryInstrument"])
df2_a, err = ek.get_data(peer_rics['Primary Instrument RIC'].tolist(), fields)
df2_a

,Instrument,Company Common Name,Country of Headquarters,Exchange Name,TRBC Industry Group Name,GICS Sector Name,Ticker Symbol,Price Currency,Company Market Cap,Enterprise Value (Daily Time Series)
0,ACN.N,Accenture PLC,Ireland; Republic of,"NEW YORK STOCK EXCHANGE, INC.",Software & IT Services,Information Technology,ACN,US Dollar,186639.950297,181147.603297
1,DELL.N,Dell Technologies Inc,United States of America,"NEW YORK STOCK EXCHANGE, INC.","Computers, Phones & Household Electronics",Information Technology,DELL,US Dollar,30293.81706,51371.81706
2,HPE.N,Hewlett Packard Enterprise Co,United States of America,"NEW YORK STOCK EXCHANGE, INC.","Computers, Phones & Household Electronics",Information Technology,HPE,US Dollar,20811.656268,31259.656268
3,MSFT.OQ,Microsoft Corp,United States of America,NASDAQ/NGS (GLOBAL SELECT MARKET),Software & IT Services,Information Technology,MSFT,US Dollar,2154162.304415,2118354.304415
4,CSCO.OQ,Cisco Systems Inc,United States of America,NASDAQ/NGS (GLOBAL SELECT MARKET),Communications & Networking,Information Technology,CSCO,US Dollar,210238.610862,197064.610862
5,INTC.OQ,Intel Corp,United States of America,NASDAQ/NGS (GLOBAL SELECT MARKET),Semiconductors & Semiconductor Equipment,Information Technology,INTC,US Dollar,135642.64356,151218.64356
6,ORCL.N,Oracle Corp,United States of America,"NEW YORK STOCK EXCHANGE, INC.",Software & IT Services,Information Technology,ORCL,US Dollar,253133.43552,336684.43552
7,CRM.N,Salesforce Inc,United States of America,"NEW YORK STOCK EXCHANGE, INC.",Software & IT Services,Information Technology,CRM,US Dollar,191490.0,190374.0
8,AMZN.OQ,Amazon.com Inc,United States of America,NASDAQ/NGS (GLOBAL SELECT MARKET),Diversified Retail,Consumer Discretionary,AMZN,US Dollar,1046962.529373,1065437.529373
9,AVGO.OQ,Broadcom Inc,United States of America,NASDAQ/NGS (GLOBAL SELECT MARKET),Semiconductors & Semiconductor Equipment,Information Technology,AVGO,US Dollar,261573.737557,284471.737557


In [18]:
rics = df2_a['Instrument'].values.tolist()
rics_lista = rics[0:2] + [ric]
rics_lista

['ACN.N', 'DELL.N', 'IBM.N']

In [21]:
from datetime import datetime, timedelta
end = datetime.today()
start = end - timedelta(days=(365.2*4))
print(start, end)

2019-04-11 18:04:22.494818 2023-04-11 13:16:22.494818


In [29]:
df = ek.get_timeseries(rics_lista[0], start_date = start, end_date = end, fields='CLOSE', interval="daily")
df[rics_lista[0]] = w['CLOSE']
daily_prices = df.drop('CLOSE', 1).iloc[1:]
daily_prices

/var/folders/h0/fd2sjrqn69ncrtc29q0mnst00000gn/T/ipykernel_1949/237603590.py:3: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



ACN.N,ACN.N
Date,
2019-04-15,137.4481
2019-04-16,138.632503
2019-04-17,132.872864
2019-04-18,134.038164
2019-04-22,132.662728
...,...
2023-04-04,131.6
2023-04-05,132.14
2023-04-06,130.5


In [30]:
dias = daily_prices.shape[0]
for i in rics_lista[1:]:
    w = ek.get_timeseries(i, start_date = start, end_date = end, fields='CLOSE', interval="daily")
    w[i] = w['CLOSE']
    w1 = w.drop('CLOSE', 1).iloc[1:]
    p = round(w1.count()[0])
    print(i, p)
    if p == dias:    
        daily_prices = daily_prices.join(w1, lsuffix="_left", rsuffix="_right")

/var/folders/h0/fd2sjrqn69ncrtc29q0mnst00000gn/T/ipykernel_1949/2522139090.py:5: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



DELL.N 1005
IBM.N 1005


/var/folders/h0/fd2sjrqn69ncrtc29q0mnst00000gn/T/ipykernel_1949/2522139090.py:5: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



In [31]:
daily_prices

,ACN.N,DELL.N,IBM.N
Date,,,
2019-04-15,137.4481,32.315039,137.4481
2019-04-16,138.632503,31.76364,138.632503
2019-04-17,132.872864,31.743405,132.872864
2019-04-18,134.038164,31.793992,134.038164
2019-04-22,132.662728,31.996341,132.662728
...,...,...,...
2023-04-04,131.6,40.87,131.6
2023-04-05,132.14,40.8,132.14
2023-04-06,130.5,40.23,130.5


In [2]:
#!conda install -y anaconda ephem
#!conda install -c -y conda-forge pystan
#!conda install -c -y conda-forge fbprophet
#!pip3 install Cmake
!pip3 install fbprophet

  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: -10
  ╰─> [8 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib
      creating build/lib/fbprophet
      creating build/lib/fbprophet/stan_model
      COMPILING THE C++ CODE FOR MODEL anon_model_dfdaf2b8ece8a02eb11f050ec701c0ec NOW.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for fbprophet
  Running setup.py clean for fbprophet


Failed to build fbprophet
  Running setup.py install for fbprophet ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for fbprophet did not run successfully.
  │ exit code: -10
  ╰─> [10 lines of output]
      running install
      /opt/homebrew/Caskroom/miniforge/base/envs/mlp/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      running build
      running build_py
      creating build
      creating build/lib
      creating build/lib/fbprophet
      creating build/lib/fbprophet/stan_model
      COMPILING THE C++ CODE FOR MODEL anon_model_dfdaf2b8ece8a02eb11f050ec701c0ec NOW.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> fbprophet

note: This is an issue wi

In [36]:
import eikon as ek
from fbprophet import Prophet
import statsmodels.api as sm
import numpy as np
import pandas as pd
import datetime
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'fbprophet'

In [ ]:
prices = pd.read_csv('../spy_top200_021221.csv', index_col=0)
prices = prices.loc['2017-01-01':'2019-12-31'].dropna(axis=1)
prices.drop(['SPY'], axis=1, inplace=True)
returns = prices.pct_change().dropna()
returns_train = returns.loc[:'2019-06-30']
returns_test = returns.loc['2019-07-01':]
prices_train = prices.loc[returns_train.index]
prices_test = prices.loc[returns_test.index]

# length of training period must be even
prices_train = prices_train.iloc[1:]
returns_train = returns_train.iloc[1:]

In [ ]:
sd = datetime.date(day = 1, month = 1, year = 2018).strftime('%d-%m-%Y')
ed = datetime.date.today().strftime('%d-%m-%Y')
a_interval = 'daily'

ric_names = ['SH-CCN-IRNOR62','SH-WUH-3HRCOIL']
rics = {}

def get_timeseries(ric):
    print(f'Getting timeseries data of {ric}')
    df = ek.get_timeseries(ric, start_date = sd, interval = a_interval)
    df.columns.values[0] = 'Value'
    df['Date'] = pd.to_datetime(df.index)    
    df['Price'] = df['Value']
    rics[ric] = df
    
    
for ric in ric_names:
    get_timeseries(ric)